In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Creutzfeldt-Jakob_Disease/GSE62699'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Integrating mRNA and miRNA Co-Expression Networks with eQTLs in the Nucleus Accumbens of Human Chronic Alcoholics"
!Series_summary	"Alcohol consumption is known to lead to gene expression changes in the brain. After performing gene co-expression network analysis (WGCNA) of genome-wide mRNA and microRNA expressions in the Nucleus Accumbens (NAc) from subjects with alcohol dependence (AD) and matched controls six mRNA and three miRNA modules significantly correlated with AD after Bonferroni correction (adj. p≤ 0.05) were identified. Cell-type-specific transcriptome analysis revealed two of the mRNA modules to be enriched for neuronal specific marker genes and downregulated in AD, whereas the remaining four were enriched for astrocyte and microglial specific marker genes and were upregulated in AD. Using gene set enrichment analysis, the neuronal specific modules were enriched for genes involved in oxidative phosphorylation, mitochondrial dysfunction

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_age = None  # define the functions when applicable

# Check for the availability of gene expression data
is_gene_available = True  # This can be inferred from the use of Affymetrix GeneChip® Human Genome U133A 2.0 (HG-U133A 2.0)

# Check for availability of variables
trait_row = None  # No explicit information about 'Creutzfeldt-Jakob_Disease'
age_row = None  # No explicit key for 'age' found
gender_row = None  # No explicit key for 'gender' found

# Define conversion functions
def convert_trait(value):
    """Convert Creutzfeldt-Jakob_Disease trait to the right binary format if needed."""
    return None  # Since trait_row is None

def convert_age(value):
    """Convert age to continuous format."""
    if ':' in value:
        val = value.split(':')[-1].strip()
        try:
            return float(val) if val else None
        except ValueError:
            return None
    return None

def convert_gender(value):
    """Convert gender to binary format."""
    if ':' in value:
        val = value.split(':')[-1].strip().lower()
        if val == 'male':
            return 1
        elif val == 'female':
            return 0
    return None

# Save cohort information
save_cohort_info('GSE62699', './preprocessed/Creutzfeldt-Jakob_Disease/cohort_info.json', is_gene_available, trait_row is not None)


A new JSON file was created at: ./preprocessed/Creutzfeldt-Jakob_Disease/cohort_info.json
